In [2]:
import pandas as pd
import pickle
import os
import glob

In [39]:

ROOTFOLDER = "/lustre1/g/geog_pyloo/05_timemachine"
PATH_PATH = "{ROOTFOLDER}/GSV/gsv_rgb/{cityabbr}/gsvmeta/gsv_path.csv"
CURATED_FOLDER = f"{ROOTFOLDER}/_curated"
TRANSFOM_FOLDER = f"{ROOTFOLDER}/_transformed"
EXFOLDER_SEG_LONG = os.path.join(CURATED_FOLDER, "c_seg_longitudinal_all")
OBJECT_SOURCE_FOLDER = "{CURATED_FOLDER}/{city_abbr}/*_objects.parquet"
PATH_TRANSFOM_FOLDER = f"{TRANSFOM_FOLDER}/t_human_attr/sel_path"
if not os.path.exists(PATH_TRANSFOM_FOLDER):
    os.makedirs(PATH_TRANSFOM_FOLDER)
PATH_SEL = "{PATH_TRANSFOM_FOLDER}/{city_abbr}.csv"


In [28]:
# load the human prediction result and only focus on images with at least one person
# city = "Hong Kong"
def get_city_path(city):
    """Get the path of the images with at least one person in the city"""
    cityabbr = city.replace(" ", "").lower()
    df_path = pd.read_csv(PATH_PATH.format(ROOTFOLDER=ROOTFOLDER, cityabbr=cityabbr))
    objfiles = glob.glob(OBJECT_SOURCE_FOLDER.format(CURATED_FOLDER=CURATED_FOLDER, city_abbr=cityabbr))
    df = pd.concat([pd.read_parquet(f) for f in objfiles])

    df_path['img'] = df_path['path'].apply(lambda x: x.split("/")[-1])
    df_path_sel = df[df['object_name']=='person'].drop_duplicates("img").merge(df_path, on='img')[['path']]

    df_path_sel = df[df['object_name']=='person'].drop_duplicates("img").merge(df_path, on='img')[['path']]

    df_path_sel.to_csv(PATH_SEL.format(PATH_TRANSFOM_FOLDER = PATH_TRANSFOM_FOLDER, city_abbr=cityabbr), 
                    index=False)
    return print(f"Done {city}")

In [45]:
results_folder = "/lustre1/g/geog_pyloo/05_timemachine/_curated/c_human_attr/hongkong"
results_files = glob.glob(f"{results_folder}/human_attr_*.pickle")
print("Number of files: ", len(results_files))

In [68]:
# remove all of these pickle files:

In [50]:
COLS_NAME = ['gender',
              'age', 
              'side',
              'glasses', 
              'hat', 
              'obs_front',
              'bags',
              'upper',
              'lower',
              'shoes',
              'idx',
            'confidence',
              'x1','y1','x2','y2']

In [60]:
# CLEAN UP RESULTS
# test_file = results_files[-1]
# load the file
def process_one_img(test_file):
  with open(test_file, 'rb') as f:
      data = pickle.load(f)
  try: 
      df = pd.concat([pd.DataFrame(data[0]['human_attr']), pd.DataFrame(data[0]['boxes'])], axis=1)
      df.columns = COLS_NAME
      df['img'] = data[0]['img']
      return df
  except:
      return test_file

In [54]:
with open(test_file, 'rb') as f:
      data = pickle.load(f)
data

[{'img': 's5IqM2vyvtH8jciDJWcm6w_180.jpg',
  'human_attr': [],
  'boxes': array([], shape=(0, 6), dtype=float32)}]

In [61]:
from tqdm import tqdm
full_result = []
error_files = []
for test_file in tqdm(results_files):
  df = process_one_img(test_file)
  if type(df) == str:
      error_files.append(df)
      if len(error_files) % 100 == 0:
          print(f"Error files: {len(error_files)}")
  else:
    full_result.append(df)
full_result = pd.concat(full_result).reset_index(drop=True)
full_result.to_csv(f"{results_folder}/human_attr_all.csv", index=False)

  1%|          | 588/106563 [00:15<1:53:26, 15.57it/s]

Error files: 100


  1%|          | 1044/106563 [00:30<1:04:21, 27.33it/s]

Error files: 200


  2%|▏         | 1735/106563 [00:48<39:07, 44.66it/s]  

...

 99%|█████████▉| 105350/106563 [40:00<00:23, 51.74it/s]

Error files: 15700


100%|█████████▉| 106045/106563 [40:15<00:10, 50.24it/s]

Error files: 15800


100%|██████████| 106563/106563 [40:26<00:00, 43.92it/s]


In [63]:
# load all cities
city_ls = pd.read_csv("../city_meta.csv")

In [64]:
header = """conda activate paddle117
module load cuda-toolkit/11.7
"""
test_img_path = """/lustre1/g/geog_pyloo/05_timemachine/_transformed/t_human_attr/sel_path/{cityabbr}.csv"""
lines = """python deploy/pipeline/pipeline_no_viz.py --config deploy/pipeline/config/infer_cfg_pphuman.yml \
                                                   --image_path={test_img_path} \
                                                   --device=gpu \
                                                   --output_dir=/lustre1/g/geog_pyloo/05_timemachine/_curated/c_human_attr/{cityabbr} \
                                                   --visual=False \
                                                   --save_result=True
                                                   """
city_to_send = city_ls['City'].unique()
batch_size = 10
for i in range(0, len(city_to_send), batch_size):
    city_batch = city_to_send[i:i+batch_size]
    with open(f"run_human_attr_{i}.sh", "a") as f:
        for city in city_batch:
            if city in ["Hong Kong", "Paris"]:
                continue
            
            cityabbr = city.replace(" ", "").lower()
            line_to_send = lines.format(test_img_path=test_img_path.format(cityabbr=cityabbr), cityabbr=cityabbr)
            f.write(line_to_send + "\n")


In [66]:
city = "Hong Kong"
cityabbr = city.replace(" ", "").lower()
test_img_path = """/lustre1/g/geog_pyloo/05_timemachine/_transformed/t_human_attr/sel_path/{cityabbr}.csv"""
lines = """python deploy/pipeline/pipeline_no_viz.py --config deploy/pipeline/config/infer_cfg_pphuman.yml \
                                                   --image_path={test_img_path} \
                                                   --device=gpu \
                                                   --output_dir=/lustre1/g/geog_pyloo/05_timemachine/_curated/c_human_attr/{cityabbr} \
                                                   --visual=False \
                                                   --save_result=True
                                                   """
line_to_send = lines.format(test_img_path=test_img_path.format(cityabbr=cityabbr), cityabbr=cityabbr)
print(line_to_send)

python deploy/pipeline/pipeline_no_viz.py --config deploy/pipeline/config/infer_cfg_pphuman.yml                                                    --image_path=/lustre1/g/geog_pyloo/05_timemachine/_transformed/t_human_attr/sel_path/hongkong.csv                                                    --device=gpu                                                    --output_dir=/lustre1/g/geog_pyloo/05_timemachine/_curated/c_human_attr/hongkong                                                    --visual=False                                                    --save_result=True
                                                   


In [67]:
# remove all parquet file in this folder


'/lustre1/g/geog_pyloo/05_timemachine/_curated/c_human_attr/hongkong'